In [2]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
import tensorflow as tf
from tensorflow.python.framework import ops
from skimage import io, transform
from keras.models import Sequential
from keras.layers import Dropout, Dense, Flatten
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils as u
from keras import optimizers

Using TensorFlow backend.


In [3]:
def PreProcessImage(path):
    img = io.imread(path)   
    resize = transform.resize(img, (224, 224))
    #io.imshow(resize)
    sample = np.asarray(resize)
    #print(sample)
    return sample

In [4]:
f = open('train.csv')

biz2label = {}
first = True

for line in f:
    if (first):
        first = False
        continue
    biz = line.split(',')[0]
    labels = line.split(',')[1].split()
    biz2label[biz] = labels

#print(biz2label)

f1 = open('train_photo_to_biz_ids.csv')

biz2photo = {}
first = True

for line in f1:
    if (first):
        first = False
        continue
    biz = line.split(',')[1][:-1]
    photo_id = line.split(',')[0]
    if biz in biz2photo:
        biz2photo[biz].append(photo_id)
    else:
        biz2photo[biz] = [photo_id]

train_biz2photo = {}
test_biz2photo = {}

c = 0
for biz, photos in biz2photo.items():
    if (c>=1032):
        break
    if (len(photos)>=32):
        if (c <1000):
            train_biz2photo[biz]=photos[:32]
        else:
            test_biz2photo[biz]=photos[:32] 
        c += 1
    
print(len(train_biz2photo))
print(len(test_biz2photo))

train_photo2label = []

for biz in sorted(train_biz2photo.keys(), key = lambda r: float(r)):
    for photo in sorted(train_biz2photo[biz], key = lambda r: float(r)):
        label = []
        for i in range(9):
            if (str(i) in biz2label[biz]):
                label.append(1)
            else:
                label.append(0)
        train_photo2label.append((photo,label))

Y_train = np.array([item[1] for item in train_photo2label])
print(Y_train.shape)

#print(photo2label)

1000
32
(32000, 9)


In [5]:
directory_str = "./train_photos/"
X_train = np.empty((32000,224,224,3))
for i in range(len(train_photo2label)):
    img_id = train_photo2label[i][0] + ".jpg"
    img_path = directory_str + img_id
    X_train[i] = PreProcessImage(img_path)
    if (i%1000==0):
        print(i)

print(X_train.shape)
    

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
(32000, 224, 224, 3)


In [6]:
X_test = np.empty((1024,224,224,3))
test_photo_ids = []
for i in range(len(test_biz2photo)):
    for k in range(32):
        photo = sorted(test_biz2photo[sorted(test_biz2photo.keys(), key = lambda r: float(r))[i]], 
                       key = lambda r: float(r))[k]
        test_photo_ids.append(photo)
        img_path = './testphotoss/' + photo + ".jpg"
        X_test[32*i+k] = PreProcessImage(img_path)
# np_a = np.array(test_photo_ids)
# np.savetxt('testphotos.txt',np_a,fmt='%s')
print(X_test.shape)

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


(1024, 224, 224, 3)


In [7]:
model = Sequential()
model.add(Conv2D(32, kernel_size = (5,5), padding = 'same', activation = 'relu', input_shape = X_train[0].shape))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Conv2D(64, (5,5), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(100, activation = 'relu'))
model.add(Dense(9, activation = 'sigmoid'))

In [8]:
#model.compile(loss=tf.keras.losses.binary_crossentropy, optimizer = tf.keras.optimizers.Adam(), metrics = ["accuracy"])
model.compile(loss='binary_crossentropy', optimizer = 'adam')

In [9]:
histories = []
accuracies = []
for i in range(5):
    history = model.fit(X_train, Y_train, batch_size = 512, epochs = 1)
    
    histories.append(history)

Epoch 1/1
32000/32000 [==============================] - 1995s 62ms/step - loss: 0.7214
Epoch 1/1
32000/32000 [==============================] - 1986s 62ms/step - loss: 0.6173
Epoch 1/1
32000/32000 [==============================] - 1986s 62ms/step - loss: 0.6080
Epoch 1/1
32000/32000 [==============================] - 1984s 62ms/step - loss: 0.5994
Epoch 1/1
32000/32000 [==============================] - 1989s 62ms/step - loss: 0.5895


In [50]:
model.fit(X_train, Y_train, batch_size = 512, epochs = 5)

Epoch 1/5
32000/32000 [==============================] - 1989s 62ms/step - loss: 0.5865
Epoch 2/5
32000/32000 [==============================] - 1986s 62ms/step - loss: 0.5796
Epoch 3/5
32000/32000 [==============================] - 1985s 62ms/step - loss: 0.5694
Epoch 4/5
32000/32000 [==============================] - 1985s 62ms/step - loss: 0.5567
Epoch 5/5
32000/32000 [==============================] - 1988s 62ms/step - loss: 0.5443


In [53]:
prediction = np.array(model.predict(X_test))

from keras.models import load_model
pred10 = np.array(load_model('basic_cnn.h5').predict(X_test))

In [54]:
b_pred = np.empty((32,9))
for k in range(32):
    b_pred[k]=(np.sum(prediction[32*k:32*(k+1)],axis=0)/32)
    
b_pred = b_pred >= 0.5
    
b_true = np.empty((32,9))

for i in range(len(test_biz2photo)):
    biz = sorted(test_biz2photo.keys(), key = lambda r: float(r))[i]
    label = []
    for k in range(9):
        if (str(k) in biz2label[biz]):
            label.append(1)
        else:
            label.append(0)
    b_true[i] = label

print(b_pred)
print(b_true)

[[False  True  True  True False  True  True  True False]
 [False  True  True  True False  True  True False False]
 [False False False  True False  True  True False  True]
 [False  True  True False False  True  True False  True]
 [False False  True  True False  True  True False  True]
 [False  True  True  True  True  True  True  True False]
 [False False False  True False  True  True False  True]
 [False  True  True  True False  True  True False  True]
 [False  True  True False False  True  True False  True]
 [False False  True  True False  True  True False  True]
 [False  True  True  True False  True  True False  True]
 [False  True  True  True False  True  True False  True]
 [False False  True  True False  True  True False  True]
 [False  True  True  True False  True  True False  True]
 [False  True  True  True False  True  True  True False]
 [False False  True False False  True  True False  True]
 [False  True  True  True False  True  True False  True]
 [False False False  True False

In [57]:
from sklearn.metrics import f1_score
from statistics import mean
f1s = []
# precision = []
# recall = []
# accuracy = []

#print(f1_score(b_true.flatten(),b_pred.flatten()))
    
for i in range(9):
    f1s.append(f1_score(b_true[:,i],b_pred[:,i]))

print(f1s)
meanf1 = mean(f1s)
print(meanf1)

0.7114093959731543
[0.0, 0.7272727272727272, 0.7692307692307693, 0.5365853658536585, 0.2222222222222222, 0.7692307692307693, 0.8571428571428571, 0.6153846153846153, 0.8636363636363636]
0.5956339655526647


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [58]:
model.save('baseline.h5')